# Class Project
## GenePattern Single Cell Analyses Workshop 

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Log in to GenePettern with your credentials.
</div>

In [21]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

For this project, we will use data from the Human Cell Atlas, for the project [Single cell transcriptome analysis of human pancreas reveals transcriptional signatures of aging and somatic mutation patterns](https://data.humancellatlas.org/explore/projects/cddab57b-6868-4be4-806f-395ed9dd635a/m/expression-matrices).  This project, and a few others, are available on the Human Cell Atlas Data Browser and you can experiment with finding other available studies on its [Explore Data Page](https://data.humancellatlas.org/explore/projects?filter=%5B%7B%22facetName%22%3A%22genusSpecies%22%2C%22terms%22%3A%5B%22Homo+sapiens%22%5D%7D%2C%7B%22facetName%22%3A%22organ%22%2C%22terms%22%3A%5B%22pancreas%22%5D%7D%5D), in this case the link (above) will pre-filter for Human Pancreas datasets to make it easy to find the experiment we will use.

From [the project page](https://data.humancellatlas.org/explore/projects/cddab57b-6868-4be4-806f-395ed9dd635a/m/expression-matrices), we can obtain the link to the file which contains the RNA-Seq data in the MTX format:
https://data.humancellatlas.org/project-assets/project-matrices/cddab57b-6868-4be4-806f-395ed9dd635a.homo_sapiens.mtx.zip 


<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
<ol>    
    <li>Drag or Copy/Paste the link to the <code>mtx.zip</code> file (above) into the <code>url*</code> parameter below.</li>
    <li>Click 'Run'</li>

</div>

In [23]:
import os 
import shutil
import urllib.request
import subprocess
import rpy2
%load_ext nbtools.r_support

@genepattern.build_ui(description="Setup the R and Python environments for the rest of this notebook. Downloads the example dataset to the notebook server.", 
                      parameters={"url":{"name":"url"},
                                  "output_var": {"name":"folder","default":"folder","hide":"true"}
                                 })
def download_MTX_zip(url):
    %load_ext rpy2.ipython
    print("Retrieving input data...") 
    base_name = 'temp_data.mtx.zip'
    os.makedirs('data/input_data/unzipped_data', exist_ok=True)
    urllib.request.urlretrieve(url, f'data/input_data/{base_name}')
    subprocess.run(["unzip", f"data/input_data/"+f"{base_name}","-d",f"data/input_data/unzipped_data/"])
    folder = os.listdir("data/input_data/unzipped_data/")[0]
    folder = os.path.join("data/input_data/unzipped_data",folder)
    
    # cache the last downloaded copy if its a repeat (just in case)
    unzipPath = "data/input_data/unzipped_data/downloaded_MTX_folder"
    if os.path.exists(unzipPath):
        if os.path.exists(unzipPath + "_PREVIOUS"):
            shutil.rmtree(unzipPath + "_PREVIOUS")
        print("Saving old copy of the data as "+unzipPath + "_PREVIOUS")
        os.rename(unzipPath, unzipPath + "_PREVIOUS");  
        
    os.rename(folder,"data/input_data/unzipped_data/downloaded_MTX_folder")
    folder = 'data/input_data/unzipped_data/downloaded_MTX_folder'
    print(f'Data unzipped to: {folder}')
    print("Done.")
    return folder

The nbtools.r_support extension is already loaded. To reload it, use:
  %reload_ext nbtools.r_support


UIBuilder(description='Setup the R and Python environments for the rest of this notebook. Downloads the exampl…

Now we have a copy of the project data as a zipped MTX file on our GenePattern Notebook server.  We need to unzip it so that Seurat can read the MTX file.


<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
Input here the directory where your data resides. If you used default parameters, it should be in `data/input_data/unzipped_data/downloaded_MTX_folder`
</div>

In [5]:
%%r_build_ui { "name": "Setup Seurat Objects", "parameters": { "data_dir":{"name":"data_dir",},"output_var": { "hide": "True" } } }

setupR <- function(data_dir){
    
    write("Loading libraries...", stdout())
    suppressMessages(library(Seurat))
    suppressMessages(library(scater))
    # Load the dataset
    write(c("Reading data from",data_dir), stdout())
    suppressMessages(pbmc.data <- Read10X(data.dir = data_dir))
    
    # Initialize the Seurat object with the raw (non-normalized data).
    write("Loadig data into Seurat...", stdout())
    pbmc <- CreateSeuratObject(counts = pbmc.data, project = "pbmc3k", min.cells = 3, min.features = 200)
    write("Done with this step.", stdout())
    return(pbmc)
}
suppressMessages(pbmc <- setupR(data_dir))

UIBuilder(function_import='nbtools._r_wrappers["4DBCBB661DDFBC2F6A2C4A9AE8A0F0C7"]', name='Setup Seurat Object…

As in the example earlier today, we will want to filter the dataset based on the mitochondrial QC metrics.  To make this sort of filtering easy we will add the metrics directly into the Seurat object.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
Click `Run` to add mitochondrial QC metrics.
</div>

In [6]:
%%r_build_ui { "name": "Add Mitochondrial QC Metrics", "parameters": { "column_name": { "type": "string", "default":"percent.mt" },"pattern": { "type": "string", "default":"MT-" }, "output_var": { "hide": "True" } } }

set_mito_qc <- function(colName, pat) {
    write("Calculating the frequency of mitochondrial genes...", stdout())
    pattern <- paste("^", trimws(pat, which = "both"), sep="")
    
    # The [[ operator can add columns to object metadata. This is a great place to stash QC stats
    pbmc[[colName]] <- PercentageFeatureSet(pbmc, pattern = pattern)
    write("Done!", stdout())
    return(pbmc)
}


suppressMessages(pbmc <- set_mito_qc(column_name, pattern))

UIBuilder(function_import='nbtools._r_wrappers["B165FAE49A60998BA866AF1CEBB11747"]', name='Add Mitochondrial Q…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>

Plot the three features (nFeature_RNA, nCount_RNA, and percent.mt) to decide which filters to use in the next step.
</div>

In [7]:
%%r_build_ui -w 800 { "width": 10, "height": 300, "name": "Triple Violin Plot", "parameters": { "first_feature": { "type": "string", "default":"nFeature_RNA" }, "second_feature":{ "type": "string", "default":"nCount_RNA"}, "third_feature": { "type": "string", "default":"percent.mt" }, "output_var":{"hide":"True"} } }
# Visualize QC metrics as a violin plot
#VlnPlot(pbmc, features = c(first_feature, second_feature, third_feature), ncol = 3)
tripleViolin <- function(first, second, third){
     
    feats <- c(first, second, third)
    plot(VlnPlot(pbmc, features = feats, ncol = 3, combine=TRUE), fig.height=5, fig.width=15)
    return("")
}

tripleViolin(first_feature, second_feature, third_feature)

UIBuilder(function_import='nbtools._r_wrappers["11F3B6CF40BC2A8D74678B35C4D2033E"]', name='Triple Violin Plot'…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>

Based on the three violin plots above, input which filters to apply to the data.
</div>

In [8]:
%%r_build_ui { "name": "Subset Data", "parameters": { "min_n_features": { "type": "number", "default":"200" },"max_n_features": { "type": "number", "default":"9000" },"max_percent_mitochondrial": { "type": "number", "default":"50" }, "output_var": { "hide": "True" } } }

my_subset <- function(min_n_features, max_n_features, max_percent_mitochondrial){
#     print(pbmc)
    pbmc <- subset(pbmc, subset = nFeature_RNA > min_n_features & nFeature_RNA < max_n_features & percent.mt < max_percent_mitochondrial)
#     print(pbmc)
    write('filtering done!', stdout())
    return(pbmc)
}

pbmc <- my_subset(min_n_features, max_n_features, max_percent_mitochonrial)

UIBuilder(function_import='nbtools._r_wrappers["A2F614CB80DF2ED9E8CA027E3C1740E9"]', name='Subset Data', origi…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Click run to apply a log-normalization and scaling of the data.
</div>

In [9]:

%%r_build_ui { "name": "Normalize", "parameters": { "method": { "type": "string", "default":"LogNormalize" },"scale_factor": { "type": "number", "default":"10000" }, "output_var": { "hide": "True" } } }

norm_pbmc <- function(meth, scale){
    write("Normalizing data...", stdout())
    invisible(pbmc <- NormalizeData(pbmc, normalization.method = meth, scale.factor = scale, verbose = F))
    write('Normalization done!', stdout())
    return(pbmc)
}

pbmc <- norm_pbmc(method, scale_factor)

UIBuilder(function_import='nbtools._r_wrappers["C50956DAB91633E16BBC8C037F9BB6FC"]', name='Normalize', origin=…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Click run to show genes which are highly variable in this experiment. Take note of these genes as they may be helpful in downstream analyses.
</div>

In [10]:
%%r_build_ui { "name": "Feature Selection", "parameters": { "method": { "type": "string", "default":"vst","hide":"True" },"num_features": { "type": "number", "default":"2000" }, "num_to_label":{"type": "number", "default": "10", "description": "label the top N features in the plot."}, "output_var": { "hide": "True" } } }
#%%R -w 800 -h 450

feat_sel_plot <- function(meth, nFeat, nLabel){
    write("Identifying variable features...", stdout())
    invisible(capture.output(pbmc <- FindVariableFeatures(pbmc, selection.method = meth, nfeatures = nFeat, 
                                                         verbose=F)))
    write("Done!", stdout())

    # Identify the 10 most highly variable genes
    top10 <- head(VariableFeatures(pbmc), nLabel)

    # plot variable features with and without labels
    invisible(capture.output(plot1 <- VariableFeaturePlot(pbmc)))
    invisible(capture.output(plot2 <- LabelPoints(plot = plot1, points = top10, repel = TRUE)))
    print(plot2)
    #plot(CombinePlots(plots = list(plot1, plot2)))
    return(pbmc)
}

pbmc <- feat_sel_plot(method, num_features, num_to_label)

UIBuilder(function_import='nbtools._r_wrappers["C03D0596B01BBF43C32F26C638F1EF43"]', name='Feature Selection',…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Click run to compute PCA.
</div>

In [11]:
%%r_build_ui {"name": "Compute PCA", "parameters": {"output_var":{"hide": "True"}}}
myscale <- function(pbmc){
    write("Scaling data...", stdout())
    all.genes <- rownames(pbmc)
    invisible(capture.output(pbmc <- ScaleData(pbmc, features = all.genes, verbose = F)))
    write('Done scaling data!', stdout())
    
    feats <- VariableFeatures(object = pbmc, verbose = F)
    pbmc <- RunPCA(pbmc, features = feats, nfeatures.print=5, verbose=F)
    write('Done computing PCA. Elbow plot below.', stdout())
    
    plot(ElbowPlot(pbmc))
    
    return(pbmc)
}
pbmc <- myscale(pbmc)

UIBuilder(function_import='nbtools._r_wrappers["E1FF552331D738F11B5812417D1A4E06"]', name='Compute PCA', origi…

Now that the PCA results have been added to the Seurat object, we want to save it again so that we can run the SeuratClustering module on the GenePattern server.  



#### Why do we run Seurat Clustering on the GenePattern server instead of in this Notebook directly?
*expand this cell to see the hidden answer*

Performing UMAP or TSNE takes more memory and CPU cycles than the simpler analyses we have been doing directly in this notebook.  By sending these to the GenePattern module, we can use the cluster behind GenePattern which has machines with more CPU and memory than our local notebook server for the few seconds or minutes the analysis requires, and not the hours that the notebook might be in use, reducing our compute costs while making the analysis run faster.

#### Save and upload the Seurat object


<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Click 'Run' to save the Seurat object as an RDS file.
</div>

In [12]:
%%r_build_ui {"name":"Save preprocessed dataset", "parameters": {  "file_name": {"default":"Seurat_preprocessed.rds"}, "output_var": {"hide": "True"} }}
save_it <- function(fileName){
    write('Saving file to the notebook workspace. This may take a while (this file may be aboyt 1 GB in size)...', stdout())
    saveRDS(pbmc, file = fileName)
    print("Saved file!")
    return(pbmc)
}
save_it(file_name)

UIBuilder(function_import='nbtools._r_wrappers["CCE3BC32EB422B0FFB95ACC9991BDBDF"]', name='Save preprocessed d…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Click 'Run' to upload the RDS file to the GenePattern server.
</div>
<div class="well well-sm">
<b>Note:</b> Note:  This can potentially take a long time as the RDS file is nearly 1GB in size.  A pre-generated file is available below if this takes too long or fails.
</div>



In [13]:
@nbtools.build_ui(name="Upload file to GenePattern Server", parameters={
    "file": {
        "name": "File to upload:",
        "default":"Seurat_preprocessed.rds"
    },
    "output_var": {
    "name": "results",
    "description": "",
    "default": "quantification_source",
    "hide": True
    }
})
def load_file(file):
    import genepattern
    uio = nbtools.UIOutput()
    display(uio)
    size = os.path.getsize(file)
    print(f'This file size is {round(size/1e6)} MB, it may take a while to upload.')
    uio.status = "Uploading..."
    uploaded_file = genepattern.session.get(0).upload_file(file_name=os.path.basename(file),file_path=file)
    uio.status = "Uploaded!"
    display(nbtools.UIOutput(files=[uploaded_file.get_url()]))
    return()

UIBuilder(function_import='nbtools.tool(id="Upload file to GenePattern Server", origin="Notebook").function_or…

### Pre-generated RDS file
*Expand this cell to access a pre-generated rds file if the save & upload is taking too long*

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
If the above steps are taking too long (save+upload), feel free to use this.
</div>

In [2]:
display(nbtools.UIOutput(text='Feel free to use this pre-generated RDS file to save time and/or disk space.',
                         files=['https://datasets.genepattern.org/data/module_support_files/SeuratClustering/Seurat_preprocessed_prebaked.rds']))

UIOutput(files=['https://datasets.genepattern.org/data/module_support_files/SeuratClustering/Seurat_preprocess…

## Seurat Clustering

Now we will cluster our pancreas data using umap.
<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    <ol>
        <li>Select your RDS file (or the pre-baked one) from the `input seurat rds file*` dropdown.</li>
        <li>Select `umap` for the `reduction` parameter.</li>
        <li>Leave all the other parameters at their defaults (or experiment with them!).</li>
        <li>Click 'Run' to start the analysis.</li>
    </ol>
</div>

In [22]:
seuratclustering_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00408')
seuratclustering_job_spec = seuratclustering_task.make_job_spec()
seuratclustering_job_spec.set_parameter("input.seurat.rds.file", "")
seuratclustering_job_spec.set_parameter("output.filename", "<input.seurat.rds.file_basename>.clustered")
seuratclustering_job_spec.set_parameter("maximum_dimension", "10")
seuratclustering_job_spec.set_parameter("resolution", "0.5")
seuratclustering_job_spec.set_parameter("reduction", "umap")
seuratclustering_job_spec.set_parameter("job.memory", "2 Gb")
seuratclustering_job_spec.set_parameter("job.queue", "gp-cloud-default")
seuratclustering_job_spec.set_parameter("job.cpuCount", "1")
seuratclustering_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(seuratclustering_task)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00408')

# Visualize Seurat results

#### pre-baked Results

If your SeuratClustering module is taking too long, you can use our pre-generated results instead of waiting for your analysis to complete.

*Expand this cell to access pre-generated SeuratClustering results instead of the results from your analysis*

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
The cell below (Job #199267) can be used to access the pre-baked results.
- Note: if you wish to use it, make sure you run it (e.g., click on the play symbol ⏯ next to it)
</div>

In [ ]:
job199267 = gp.GPJob(genepattern.session.get(0), 199267)
genepattern.display(job199267)

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Download the CSV file.
</div>

## Download files from GPServer

Since we ran the clustering on the GenePattern server instead of the notebook, now we will bring them back down to the notebook server so that we can visualize them within the notebook.  We need to download both the CSV and the RDS files.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
To download the CSV file, choose from the dropdown menu the CSV file that the SeuratClustering module outputs (it should end with `*.clustered.csv`)

- Alternatively, chose the CSV file which includes the term `prebaked` in it.
</div>
<div class="well well-sm">
<b>Note:</b> This takes ~ 1 second.
</div>

In [14]:
import os

DownloadJobResultFile_description = "Download file from a GenePattern module job result."
DownloadJobResultFile_parameters = {"file": {"type": "file", "kinds": ["csv", "rds"]}, "output_var": {"hide": True}}   
        
def DownloadJobResultFile(file):
    # extract job number and file name from url
    job_num = file.split("/")[-2]
    remote_file_name = file.split("/")[-1]
    
    # get the job based on the job number passed as the second argument
    job = gp.GPJob(genepattern.get_session(0), job_num)

    # fetch a specific file from the job
    remote_file = job.get_file(remote_file_name)
    
    uio = nbtools.UIOutput(text=file)
    display(uio)
    uio.status = "Downloading..."
    
    File_Name = os.path.basename(file)

    response = remote_file.open()
    CHUNK = 16 * 1024
    with open(File_Name, 'wb') as f:
        while True:
            chunk = response.read(CHUNK)
            if not chunk:
                break
            f.write(chunk)
    uio.status = "Downloaded!"
    print(File_Name)
    display(nbtools.UIOutput(files=[File_Name]))
    
genepattern.GPUIBuilder(DownloadJobResultFile, collapse=False,
                    name='Download File From GenePattern Server',
                    description=DownloadJobResultFile_description,
                    parameters=DownloadJobResultFile_parameters)

UIBuilder(collapse=False, description='Download file from a GenePattern module job result.', function_import='…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
To download the RDS file, choose from the dropdown menu the CSV file that the SeuratClustering module outputs (it should end with `*.clustered.rds`)

- Alternatively, chose the RDS file which includes the term `prebaked` in it.
</div>
<div class="well well-sm">
<b>Note:</b> This takes about 30 seconds on a <i>high speed</i> connnection, please be patient.
</div>

In [15]:
genepattern.GPUIBuilder(DownloadJobResultFile, collapse=False,
                    name='Download File From GenePattern Server',
                    description=DownloadJobResultFile_description,
                    parameters=DownloadJobResultFile_parameters)

UIBuilder(collapse=False, description='Download file from a GenePattern module job result.', function_import='…

## Load data into notebook

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
    
Click 'Run' to load the results you just downloaded.
- Note: make sure you type the name of the files downloaded in the two steps above.
</div>

In [16]:
%%r_build_ui {"name":"Load dataset with clustering", "parameters": {"RDS_url":{"name":"RDS file","default":"Seurat_preprocessed.clustered.rds.rds",'type':"file"}, "CSV_url":{"name":"CSV_file","type":"file","default":"Seurat_preprocessed.clustered.rds.csv"}, "output_var": {"hide": "True"} }}

load_markers <- function(CSV_url) {
    write("Loading cluster markers into notebook...", stdout())
    markers <- read.csv(CSV_url)
    write("Done!", stdout())
    return(markers)
}
load_it <- function(RDS_url){
    write("Loading clustering results into notebook...", stdout())
    pbmc <- readRDS(file = RDS_url)
    write("Loaded file!", stdout())
    return(pbmc)
}
suppressWarnings(markers <- load_markers(CSV_url))
pbmc <- load_it(RDS_url)

UIBuilder(function_import='nbtools._r_wrappers["735F91BEF0D56CEBB57050B98F328A34"]', name='Load dataset with c…

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
Click 'Run' to plot the results of the umap clustering.
</div>

In [17]:
%%r_build_ui {"name": "Visualize clusters", "parameters": {"output_var": {"hide": "True"}}}
library(Seurat)
do_dim_plot <- function() {
    plot(DimPlot(pbmc, reduction = "umap"))
    return("")
}
do_dim_plot()

UIBuilder(function_import='nbtools._r_wrappers["100D63F9A66366946C781582B4474DC2"]', name='Visualize clusters'…

We want to see which genes are markers for the clusters we see above.  Lets look at the markers

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
<ol>    
    <li>Select one of the clusters (e.g. 1,2,3).</li>
    <li>Click 'Run' to show the top markers for a cluster.</li>
    </ol>
</div>

In [18]:
%%r_build_ui {"name": "Print top cluster markers", "parameters": {"cluster_number": {}, "output_var": {"hide": "True"}}}
print_top_markers <- function(cluster_number) {
    return(head(markers[markers$cluster==cluster_number,], n = 5))
}
print_top_markers(cluster_number)

UIBuilder(function_import='nbtools._r_wrappers["077F3BBF37196AFC5B0780EE6D3BC892"]', name='Print top cluster m…

And now lets look at the gene expression for one of these markers.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
<ol>    
    <li>Enter the gene name for a cluster marker.</li>
    <li>Click 'Run' to show a violin plot of its gene expression.</li>
    <li>Repeat as desired for several marker genes.</li>
    </ol>
</div>

In [19]:
%%r_build_ui {"name": "Violin plot of gene expression", "parameters": {"gene": {}, "output_var": {"hide": "True"}}}
do_violin <- function(gene) {
    plot(VlnPlot(pbmc, features = c(gene), slot = "counts", log = TRUE))
    return("")
}
do_violin(gene)

UIBuilder(function_import='nbtools._r_wrappers["8C454A64890C2C7A2D922E865D1C224E"]', name='Violin plot of gene…

And now lets look at the gene expression for this marker across all clusters.

<div class="alert alert-info">
<p class="lead"> Instructions <i class="fa fa-info-circle"></i></p>
<ol>    
    <li>Enter the gene name for a cluster marker.</li>
    <li>Click 'Run'.</li>
    </ol>
</div>

In [20]:
%%r_build_ui {"name": "Expression of gene across all clusters", "parameters": {"gene": {}, "output_var": {"hide": "True"}}}
do_umap_gene <- function(gene) {
    plot(FeaturePlot(pbmc, features = c(gene)))
    return("")
}
do_umap_gene(gene)

UIBuilder(function_import='nbtools._r_wrappers["DB26B770F9264833316D13505C258D21"]', name='Expression of gene …

<div class="well well-sm">
    
Finally, take a look at a few different genes, some suggestions are: MMP7, and MMP2.
- Crawford, Howard C et al. “Matrix metalloproteinase-7 is expressed by pancreatic cancer precursors and regulates acinar-to-ductal metaplasia in exocrine pancreas.” The Journal of clinical investigation vol. 109,11 (2002): 1437-44. doi:10.1172/JCI15051
- Jakubowska, et al. “Expressions of Matrix Metalloproteinases 2, 7, and 9 in Carcinogenesis of Pancreatic Ductal Adenocarcinoma.” Disease Markers, Hindawi, 26 June 2016, www.hindawi.com/journals/dm/2016/9895721/.
    
</div>